## Intro

A pointer is a general concept for a variable that contains an address in memory. This address refers to, or “points at,” some other data.  <span style="color:lightgreen">***The most common kind of pointer in Rust is a reference** (Chapter 4). References are indicated by the `&` symbol and borrow the value they point to. They don’t have any special capabilities other than referring to data, and have no overhead.*</span>

<span style="color:lightgreen">*Smart pointers, on the other hand, are data structures that act like a pointer but also have additional metadata and capabilities (like in C++)*</span>. Rust, with its concept of ownership and borrowing, has an additional difference between references and smart pointers: while references only borrow data, in many cases, smart pointers own the data they point to. *For example, `String` and `Vec<T>` are smart pointers because they own some memory and allow you to manipulate it. They also have metadata and extra capabilities or guarantees.*

<span style="color:lightgreen">*Smart pointers are usually implemented using structs, but they specifically implements the `Deref` and `Drop` traits*</span>. The `Deref` trait allows an instance of the smart pointer struct to behave like a reference so you can write your code to work with either references or smart pointers. The `Drop` trait allows you to customize the code that’s run when an instance of the smart pointer goes out of scope.

In this chapter, we will cover the most common types of smart pointers:
- `Box<T>` for allocating values on the heap
- `Rc<T>`, a reference counting type that enables multiple ownership
- `Ref<T>` and `RefMut<T>`, accessed through `RefCell<T>`, a type that enforces the borrowing rules at runtime instead of compile time

## Using `Box<T>` to Point to Data on the Heap

The most straightforward smart pointer is a box, whose type is written `Box<T>`. <span style="color:lightgreen">*`Box<T>` allow you to store data on the heap rather than the stack. What remains on the stack is the pointer to the heap data.*</span>

Boxes don’t have performance overhead, other than storing their data on the heap instead of on the stack. But they don’t have many extra capabilities either. You’ll use them most often in these situations:

- When you have a type whose size can’t be known at compile time and you want to use a value of that type in a context that requires an exact size
- When you have a large amount of data and you want to transfer ownership but ensure the data won’t be copied when you do so, so we store the large data on the heap in a box and only the small amount of pointer data is copied around on the stack, while the data it references stays in one place on the heap.
- When you want to own a value and you care only that it’s a type that implements a particular trait rather than being of a specific type


In [3]:
fn main() {
    let b = Box::new(5);  //  b points to the value `5` which stays on the heap
    println!("b = {}", b);
}

// Just like any owned value, when a box goes out of scope, as b does at the end 
// of main, it will be deallocated. The deallocation happens both for the box 
// (stored on the stack) and the data it points to (stored on the heap).

main()

b = 5


()

### Enabling Recursive Types with Boxes

First attempt to define a recursive `con` list

In [5]:
enum List {
    Cons(i32, List),
    Nil,
}

use crate::List::{Cons, Nil};

fn main() {
    let list = Cons(1, Cons(2, Cons(3, Nil)));
}

main()

Error: recursive type `List` has infinite size

The error shows this type “has infinite size.” The reason is that we’ve defined `List` with a variant that is recursive: it holds another value of itself directly. As a result, Rust can’t figure out how much space it needs to store a `List` value. Let’s break down why we get this error

### How Rust Computes the Size of a Non-Recursive Type

For example, in the `Message` struct below, to determine how much space to allocate for a `Message` value, Rust goes through each of the variants to see which variant needs the most space, since only one variant will be used

In [6]:
enum Message {
    Quit,  // no need for any space
    Move { x: i32, y: i32 },  // needs enough space to store two i32 values
    Write(String),  // space for a String
    ChangeColor(i32, i32, i32),  // space for three i32 values
}

fn main() {}

However, with the recursive `list = Cons(1, Cons(2, Cons(3, Nil)));`, Rust needs to gives an amount of an `i32` plus the size of a `List` for each recursion, which can continue forever

![conlist](../images/9.svg)


### Using `Box<T>` to Get a Recursive Type with a Known Size

<span style="color:lightgreen">*Because a `Box<T>` is a pointer, Rust always knows how much space a `Box<T>` needs: a pointer’s size doesn’t change based on the amount of data it’s pointing to. This means we can put a `Box<T>` inside the `Cons` variant instead of another `List` value directly. The `Box<T>` will point to the next `List` value that will be on the heap rather than inside the `Cons` variant*</span>


In [6]:
enum List {
    Cons(i32, Box<List>),
    Nil,
}

use crate::List::{Cons, Nil};

fn main() {
    let list = Cons(1, Box::new(Cons(2, Box::new(Cons(3, Box::new(Nil))))));
}

main()

()

![](../images/10.svg)


## The `Deref` Trait: Treating Smart Pointers Like Regular References

<span style="color:lightgreen">***Implementing the `Deref` trait allows you to customize the behavior of the dereference operator `*`***</span>. By implementing `Deref` in such a way that a smart pointer can be treated like a regular reference, you can write code that operates on references and use that code with smart pointers too.

### Following the Pointer to the Value

<span style="color:lightgreen">*A regular reference is a type of pointer, and one way to think of a pointer is as an arrow to a value stored somewhere else*</span>. Below we use dereference to follow the reference to the value:

In [8]:
fn main() {
    let x =5;
    let y = &x;

    assert_eq!(5, x);
    assert_eq!(5, *y);
}

main()

()

### Using `Box<T>` Like a Reference

In [9]:
fn main() {
    let x = 5;
    let y = Box::new(x);

    assert_eq!(5, x);
    assert_eq!(5, *y);
}

main()

()

The main difference between is that here we set `y` to be an instance of a `Box<T>` pointing to a copied value of `x` rather than a reference pointing to the value of `x`. However, we can use `*` on both in the same way

### Defining Our Own Smart Pointer

Below, we define the `MyBox` struct that works with any generic type `T`, and its `DeRef` trait to allows it to work with the deference operator

In [2]:
use std::ops::Deref;

impl<T> Deref for MyBox<T> {
    type Target = T;

    fn deref(&self) -> &Self::Target {
        &self.0
    }
}

struct MyBox<T>(T);

impl<T> MyBox<T> {
    fn new(x: T) -> MyBox<T> {
        MyBox(x)
    }
}

fn main() {
    let x = 5;
    let y = MyBox::new(x);

    assert_eq!(5, x);
    assert_eq!(5, *y);
}

main()

()

Things to note:
- The type `Target = T;` syntax defines an *associated type* for the `Deref` trait to use.
- We fill in the body of the `deref` method with `&self.0` so `deref` returns a reference to the value we want to access with the `*` operator
- When we entered `*y`, Rust actually does `*(y.deref())`
- The reason the `deref()` method returns a reference to a value, and that the plain dereference outside the parentheses in `*(y.deref())` is still necessary, is to do with the ownership system. If the `deref` method returned the value directly instead of a reference to the value, the value would be moved out of `self`. We don’t want to take ownership of the inner value inside `MyBox<T>` in this case or in most cases where we use the dereference operator.

### Implicit Deref Coercions with Functions and Methods

<span style="color:lightgreen">*`Deref` coercion converts a reference to a type that implements the `Deref` trait into a reference to another type*</span>.  For example, `deref` coercion can convert `&String` to `&str` because `String` implements the `Deref` trait such that it returns `&str`. `Deref` coercion is a convenience Rust performs on arguments to functions and methods, and works only on types that implement the `Deref` trait. It happens automatically when we pass a reference to a particular type’s value as an argument to a function or method that doesn’t match the parameter type in the function or method definition.

<span style="color:lightgreen">**Why? `Deref` coercion was added to Rust so that programmers writing function and method calls don’t need to add as many explicit references and dereferences with `&` and `*`.**</span>

Example: Below, we call the `hello` function with the argument `&m`, which is a reference to a `MyBox<String>` value. Because we implemented the `Deref` trait on `MyBox<T>`, Rust can turn `&MyBox<String>` into `&String` by calling `deref`. 

In [ ]:
use std::ops::Deref;

impl<T> Deref for MyBox<T> {
    type Target = T;

    fn deref(&self) -> &T {
        &self.0
    }
}

struct MyBox<T>(T);

impl<T> MyBox<T> {
    fn new(x: T) -> MyBox<T> {
        MyBox(x)
    }
}


fn hello(name: &str) {
    println!("Hello, {name}!");
}

fn main() {
    let m  = MyBox::new(String::from("Rust"));
    hello(&m);
}

If Rust does not implement deref coercion, we will have to write `hello(&(*m)[..]);`. The `(*m)` dereferences the `MyBox<String>` into a `String`. Then the `&` and `[..]` take a string slice of the `String` that is equal to the whole string to match the signature of `hello`.

### How Deref Coercion Interacts with Mutability

Similar to how you use the `Deref` trait to override the `*` operator on immutable references, you can use the `DerefMut` trait to override the `*` operator on mutable references.

Rust does deref coercion when it finds types and trait implementations in three cases:
- From `&T` to `&U` when `T`: `Deref<Target=U>`
- From `&mut T` to `&mut U` when `T: DerefMut<Target=U>`
- From `&mut T` to `&U` when `T: Deref<Target=U>` - Rust will also coerce a mutable reference to an immutable one. But the reverse is not possible: immutable references will never coerce to mutable references.

## The `Drop` Trait: Running Code on `CleanUp` 

<span style="color:lightgreen">*The second trait important to the smart pointer pattern is `Drop`, which lets you customize what happens when a value is about to go out of scope*</span>. You can provide an implementation for the `Drop` trait on any type, and that code can be used to release resources like files or network connections.

In some languages, for some types, the programmer must call code to free memory or resources every time they finish using an instance of those types. Examples include file handles, sockets, or locks. If they forget, the system might become overloaded and crash. <span style="color:lightgreen">*In Rust, you can specify that a particular bit of code be run whenever a value goes out of scope defined by the `Drop` trait, and the compiler will insert this code automatically.*</span>

In [3]:
struct CustomSmartPointer {
    data: String,
}

impl Drop for CustomSmartPointer {
    fn drop(&mut self) {
        println!("Dropping CustomSmartPointer with data `{}`!", self.data);
    }
}

fn main() {
    let c = CustomSmartPointer {
        data: String::from("my stuff"),
    };
    let d = CustomSmartPointer {
        data: String::from("other stuff"),
    };
    println!("CustomSmartPointers created.");
}

main()

CustomSmartPointers created.
Dropping CustomSmartPointer with data `other stuff`!
Dropping CustomSmartPointer with data `my stuff`!


()

### Dropping a Value Early with `std::mem::drop`

Rust doesn’t let you call the `Drop` trait’s `drop` method manually since Rust will still call it on the value at the end of of its scope. This would cause a double free error because Rust would be trying to clean up the same value twice.

In [4]:
struct CustomSmartPointer {
    data: String,
}

impl Drop for CustomSmartPointer {
    fn drop(&mut self) {
        println!("Dropping CustomSmartPointer with data `{}`!", self.data);
    }
}

fn main() {
    let c = CustomSmartPointer {
        data: String::from("some data"),
    };
    println!("CustomSmartPointer created.");
    c.drop();
    println!("CustomSmartPointer dropped before the end of main.");
}

main()

Error: explicit use of destructor method

Instead you have to call the `std::mem::drop` function provided by the standard library if you want to force a value to be dropped before the end of its scope.

In [5]:
struct CustomSmartPointer {
    data: String,
}

impl Drop for CustomSmartPointer {
    fn drop(&mut self) {
        println!("Dropping CustomSmartPointer with data `{}`!", self.data);
    }
}

fn main() {
    let c = CustomSmartPointer {
        data: String::from("some data"),
    };
    println!("CustomSmartPointer created.");
    drop(c);
    println!("CustomSmartPointer dropped before the end of main.");
}
main()

CustomSmartPointer created.
Dropping CustomSmartPointer with data `some data`!
CustomSmartPointer dropped before the end of main.


()

## `Rc<T>`: The Reference Counted Smart Pointer

In the majority of cases, ownership is clear: you know exactly which variable owns a given value. <span style="color:lightgreen">*However, there are cases when a single value might have multiple owners*</span>. For example, in graph data structures, multiple edges might point to the same node, and that node is conceptually owned by all of the edges that point to it. A node shouldn’t be cleaned up unless it doesn’t have any edges pointing to it and so has no owners.

<span style="color:lightgreen">***You have to enable multiple ownership explicitly by using the Rust type `Rc<T>`, which is an abbreviation for reference counting. The `Rc<T>` type keeps track of the number of references to a value to determine whether or not the value is still in use. If there are zero references to a value, the value can be cleaned up without any references becoming invalid.***</span>

## `RefCell<T>` and the Interior Mutability Pattern

<span style="color:lightgreen">***Interior mutability is a design pattern in Rust that allows you to mutate data even when there are immutable references to that data; normally, this action is disallowed by the borrowing rules.***</span> To mutate data, the pattern uses `unsafe` code inside a data structure to bend Rust’s usual rules that govern mutation and borrowing. Unsafe code indicates to the compiler that we’re checking the rules manually instead of relying on the compiler to check them for us

## Memory Leak with Reference Cycles 

<span style="color:lightgreen">***Rust’s memory safety guarantees make it difficult, but not impossible, to accidentally create memory that is never cleaned up (known as a memory leak)***</span>. Preventing memory leaks entirely is not one of Rust’s guarantees, meaning memory leaks are memory safe in Rust. We can see that <span style="color:lightgreen">*Rust allows memory leaks by using `Rc<T>` and `RefCell<T>`: it’s possible to create references where items refer to each other in a cycle. This creates memory leaks because the reference count of each item in the cycle will never reach 0, and the values will never be dropped.*</span>